In [0]:
%run /Users/harshith.r@diggibyte.com/Posexpo/bronze_to_silver/custom_schema

##Modules Needed

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import re

## Function to create DataFrame

In [0]:
def create_dataframe(path,format_name,schema):
    return spark.read.format(format_name).option("multiline","true").schema(schema).load(path)

## Function to Change the DataType of column in DataFrame

In [0]:
def type_cast(df,col_name,cast_type):
    return df.withColumn(col_name,col(col_name).cast(cast_type))


## Convert To Array Type

In [0]:
def explode_split(df,columnnames,deliemeter):
    return df.withColumn(columnnames, split(df[columnnames], deliemeter))

##Flatten The Array

In [0]:
def posexplode_array(df,col_name):
    pos_col_name = f"pos_{col_name}"
    explode_col_name = f"{col_name}_col"
    new_columns=[pos_col_name,explode_col_name]
    return df.select('*',posexplode(col_name).alias(*new_columns)).drop(col_name)

##Write as table

In [0]:
def write(df,tableName):
    df.write.saveAsTable(tableName)

##Convert to Date Time formate

In [0]:
def date_format(df,col_name):
    return df.withColumn(col_name, to_date(col(col_name), 'EEE MMM dd HH:mm:ss zzz yyyy'))

##Convert the Column Names To Snake Case

In [0]:
def convert_to_snake_case(column_name):
    return re.sub(r'\s+|/+', '_', column_name).lower()

def rename_all_columns_to_snake_case(df):
    new_columns = [col(column).alias(convert_to_snake_case(column)) for column in df.columns]
    return df.select(*new_columns)

##To Drop The Table

In [0]:
def drop_colum(df,*colname):
    return df.drop(*colname) 

##Filter Based On Column

In [0]:
def filter_on_pos(df, columns):
    condition = (col(columns[0]) == col(columns[1])) & (col(columns[1]) == col(columns[2]))
    filtered_df = df.filter(condition)
    return filtered_df